## Data and Library Load

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/MyDrive/Portofolio/smart_grid_stab

/content/drive/MyDrive/Portofolio/smart_grid_stab


In [3]:
!ls

Callbacks		 representative_labels.csv
model_quant.ipynb	 smart_grid_stability_augmented.csv
Models			 smart_grid_stability_train.ipynb
representative_data.csv  TFLite_Models


In [4]:
!ls /content/drive/MyDrive/Portofolio/smart_grid_stab/Models

model1	model2	model3	model4	model5	model6


In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
import glob

import tensorflow as tf


In [6]:
model = tf.keras.models.load_model('Models/model3')
model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_29 (Dense)            (None, 256)               3328      
                                                                 
 batch_normalization_9 (Batc  (None, 256)              1024      
 hNormalization)                                                 
                                                                 
 dropout_8 (Dropout)         (None, 256)               0         
                                                                 
 dense_30 (Dense)            (None, 128)               32896     
                                                                 
 batch_normalization_10 (Bat  (None, 128)              512       
 chNormalization)                                                
                                                                 
 dropout_9 (Dropout)         (None, 128)              

In [7]:
X_test = pd.read_csv("representative_data.csv") 
X_test.head()

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4
0,1.347360,-0.765822,-0.910389,-1.192113,0.320451,0.003686,0.195272,-0.754046,1.563270,0.119907,-0.080848,0.149504
1,0.220956,1.679136,-1.241222,-0.248694,-1.675419,1.657664,0.291911,0.953829,-0.134880,-0.672706,0.551318,-1.424976
2,-0.916888,-1.293003,-1.331651,-1.329583,1.719103,-1.395985,0.083366,-1.666063,1.466270,1.257912,0.264974,-1.039676
3,1.152211,0.655809,-1.170751,-0.123448,0.172703,-0.083205,0.778455,-0.993445,-0.092881,1.344447,-0.603063,-0.983731
4,0.931419,1.580810,-0.348958,-1.485702,0.109953,0.141723,-1.668924,1.334284,-1.035258,-0.594951,1.337576,-0.385165


In [8]:
y_test = pd.read_csv("representative_labels.csv") 
y_test.head()

,stabf
0,1
1,1
2,1
3,0
4,1


In [9]:
X_t =X_test.values.astype('float32')
y_t =y_test.values.astype('float32').flatten()

## Convert to TF-LITE + Quantization

https://www.tensorflow.org/lite/performance/post_training_quantization

In [10]:
model.evaluate(X_t,y_t,)#baseline

204/204 [==============================] - 1s 2ms/step - loss: 0.0675 - accuracy: 0.9797


[0.0675126165151596, 0.9797452688217163]

### No Quant

In [ ]:
# Convert the model to the TensorFlow Lite format without quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
open("TFLite_Models/model.tflite","wb").write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpk9bam48e/assets


181084

In [ ]:
!apt-get update && apt-get -qq install xxd
!xxd -i TFLite_Models/model.tflite > TFLite_Models/model.h

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [806 kB]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:13 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelea

In [ ]:
print("Model in Mb:", os.path.getsize("TFLite_Models/model.h") / float(2**20))

Model in Mb: 1.0650529861450195


In [ ]:
interpreter = tf.lite.Interpreter(model_path="TFLite_Models/model.tflite")
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
interpreter.allocate_tensors()

In [ ]:
interpreter.set_tensor(input_details[0]['index'], [X_t[0]])
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])

output_data

array([[0.97291076]], dtype=float32)

In [ ]:
#inference 
y_pred = []

for row in range(X_t.shape[0]):
  interpreter.set_tensor(input_details[0]['index'], [X_t[row]])
  interpreter.invoke()
  output_data = interpreter.get_tensor(output_details[0]['index'])

  y_pred.append(output_data)


In [ ]:
y_pred = np.array(y_pred).flatten()
y_pred[y_pred>0.5] = 1 
y_pred[y_pred<=0.5] = 0 

In [ ]:
acc = (y_pred==y_t).sum()/y_t.shape[0]
acc

0.9797452815712752

### Full integer quantization

To quantize the variable data (such as model input/output and intermediates between layers), you need to provide a RepresentativeDataset. This is a generator function that provides a set of input data that's large enough to represent typical values. It allows the converter to estimate a dynamic range for all the variable data. (The dataset does not need to be unique compared to the training or evaluation dataset.) To support multiple inputs, each representative data point is a list and elements in the list are fed to the model according to their indices.

Now all weights and variable data are quantized, and the model is significantly smaller compared to the original TensorFlow Lite model.

However, to maintain compatibility with applications that traditionally use float model input and output tensors, the TensorFlow Lite Converter leaves the model input and output tensors in float:

https://www.tensorflow.org/lite/performance/post_training_integer_quant#convert_using_float_fallback_quantization

In [155]:
def representative_data_gen():
    for i_value in tf.data.Dataset.from_tensor_slices(X_t).batch(1).take(117):
      i_value_f32 = tf.dtypes.cast(i_value,tf.float32)
      yield [i_value_f32]


In [156]:
# Convert the model to the TensorFlow Lite format without quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
tflite_model = converter.convert()

open("TFLite_Models/model.tflite","wb").write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpq4ikcazl/assets


INFO:tensorflow:Assets written to: /tmp/tmpq4ikcazl/assets
/usr/local/lib/python3.7/dist-packages/tensorflow/lite/python/convert.py:746: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


49600

In [157]:
!apt-get update && apt-get -qq install xxd
!xxd -i TFLite_Models/model.tflite > TFLite_Models/model.h

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 252 kB in 2s (106 kB/s)
Reading package lis

In [158]:
print("Model in Mb:", os.path.getsize("TFLite_Models/model.h") / float(2**20))

Model in Mb: 0.2917957305908203


In [159]:
interpreter = tf.lite.Interpreter(model_path="TFLite_Models/model.tflite")
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
interpreter.allocate_tensors()

In [160]:
interpreter.set_tensor(input_details[0]['index'], [X_t[0]])
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])

output_data

array([[0.97265625]], dtype=float32)

In [161]:
#inference 
y_pred = []

for row in range(X_t.shape[0]):
  interpreter.set_tensor(input_details[0]['index'], [X_t[row]])
  interpreter.invoke()
  output_data = interpreter.get_tensor(output_details[0]['index'])

  y_pred.append(output_data)


In [162]:
y_pred = np.array(y_pred).flatten()
y_pred[y_pred>0.5] = 1 
y_pred[y_pred<=0.5] = 0 

acc = (y_pred==y_t).sum()/y_t.shape[0]
acc

0.9800521712444377